# Calculation of MLB Statistics (wOBA, wRAA, wRC)
This file consists the code to calculate wOBA, wRAA, and wRC manually since FanGraphs for these stats are hidden behind a paywall. Note that these are not qualified batters but instead just batters with at least 1 PA. The reason for this is because we want to see how these batters do on an aggregate basis rather than on an average since we're looking at wOBA, wRAA, and wRC.

## Aggregating MLB Data from StatCast

In [1]:
import pandas as pd
import numpy as np

from pathlib import Path

In [ ]:
df_2015 = pd.read_csv("data/statcast-major-leagues/mlb-stats-2015.csv")
df_2016 = pd.read_csv("data/statcast-major-leagues/mlb-stats-2016.csv")
df_2017 = pd.read_csv("data/statcast-major-leagues/mlb-stats-2017.csv")
df_2018 = pd.read_csv("data/statcast-major-leagues/mlb-stats-2018.csv")
df_2019 = pd.read_csv("data/statcast-major-leagues/mlb-stats-2019.csv")
df_2020 = pd.read_csv("data/statcast-major-leagues/mlb-stats-2020.csv")
df_2021 = pd.read_csv("data/statcast-major-leagues/mlb-stats-2021.csv")
df_2022 = pd.read_csv("data/statcast-major-leagues/mlb-stats-2022.csv")
df_2023 = pd.read_csv("data/statcast-major-leagues/mlb-stats-2023.csv")
df_2024 = pd.read_csv("data/statcast-major-leagues/mlb-stats-2024.csv")
df_2025 = pd.read_csv("data/statcast-major-leagues/mlb-stats-2025.csv")

df_mlb_stats = pd.concat(
    [
        df_2015,
        df_2016,
        df_2017,
        df_2018,
        df_2019,
        df_2020,
        df_2021,
        df_2022,
        df_2023,
        df_2024,
        df_2025,
    ]
)


# Column name fixing
df_mlb_stats.rename(
    columns={
        "last_name, first_name": "Name",
        "player_id": "StatCast_ID",
        "year": "Year",
        "player_age": "Age",
        "ab": "AB",
        "pa": "PA",
        "single": "1B",
        "double": "2B",
        "triple": "3B",
        "home_run": "HR",
        "strikeout": "K",
        "walk": "BB",
        "k_percent": "K%",
        "bb_percent": "BB%",
        "batting_avg": "BA",
        "slg_percent": "SLG",
        "on_base_percent": "OBP",
        "on_base_plus_slg": "OPS",
        "isolated_power": "ISO",
        "babip": "BABIP",
        "r_total_caught_stealing": "CS",
        "r_total_stolen_bases": "SB",
        "b_hit_by_pitch": "HBP",
        "b_intent_walk": "IBB",
        "b_sac_fly": "SF",
        "woba": "wOBA",
        "sprint_speed": "Speed",
    },
    inplace=True,
)


# Fix name format from "last_name, first_name" to first name last name"
def fix_name(name: str) -> str:
    last, first = [part.strip() for part in name.split(",", 1)]
    return f"{first} {last}"


# Display
df_mlb_stats

,Name,StatCast_ID,Year,Age,AB,PA,1B,2B,3B,HR,...,OPS,ISO,BABIP,CS,r_total_stolen_base,HBP,IBB,SF,wOBA,Speed
0,"Ruiz, Carlos",434563,2015,36,284,320,44,13,1,2,...,0.575,0.074,0.242,1,1,4,2,1,0.259,25.6
1,"García, Avisaíl",541645,2015,24,553,601,110,17,2,13,...,0.674,0.108,0.320,7,7,8,3,4,0.295,29.0
2,"Dykstra, Allan",488852,2015,28,31,38,3,0,0,1,...,0.515,0.097,0.167,0,0,1,0,0,0.251,23.4
3,"Nelson, Jimmy",519076,2015,26,55,59,5,1,0,0,...,0.252,0.018,0.286,0,0,0,0,0,0.113,23.5
4,"Garza, Matt",490063,2015,31,39,47,3,0,0,0,...,0.177,0.000,0.167,1,0,0,0,0,0.083,23.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
619,"Steer, Spencer",668715,2025,27,472,524,73,20,2,17,...,0.704,0.159,0.279,1,7,4,0,2,0.309,28.0
620,"Díaz, Yandy",650490,2025,33,557,618,112,28,1,24,...,0.837,0.183,0.315,1,3,6,3,5,0.360,26.1
621,"Herrera, Jose",645444,2025,28,166,204,23,6,0,2,...,0.544,0.072,0.242,0,0,1,0,3,0.252,24.8
622,"Raley, Luke",670042,2025,30,170,203,23,7,0,4,...,0.627,0.112,0.275,1,2,11,0,1,0.288,28.7


## Get annual league wOBA values
Obtained from [FanGraphs](https://www.fangraphs.com/tools/guts?type=cn).

In [14]:
# wOBA scaling from 2015 to 2025 as of Sept 16th 2025
woba_values = {
    "Season": [2025, 2024, 2023, 2022, 2021, 2020, 2019, 2018, 2017, 2016, 2015],
    "wOBA": [
        0.314,
        0.310,
        0.318,
        0.310,
        0.314,
        0.320,
        0.320,
        0.315,
        0.321,
        0.318,
        0.313,
    ],
    "wOBAScale": [
        1.230,
        1.242,
        1.204,
        1.259,
        1.209,
        1.185,
        1.157,
        1.226,
        1.185,
        1.212,
        1.251,
    ],
    "wBB": [
        0.693,
        0.689,
        0.696,
        0.689,
        0.692,
        0.699,
        0.690,
        0.690,
        0.693,
        0.691,
        0.687,
    ],
    "wHBP": [
        0.724,
        0.720,
        0.726,
        0.720,
        0.722,
        0.728,
        0.719,
        0.720,
        0.723,
        0.721,
        0.718,
    ],
    "w1B": [
        0.883,
        0.882,
        0.883,
        0.884,
        0.879,
        0.883,
        0.870,
        0.880,
        0.877,
        0.878,
        0.881,
    ],
    "w2B": [
        1.252,
        1.254,
        1.244,
        1.261,
        1.242,
        1.238,
        1.217,
        1.247,
        1.232,
        1.242,
        1.256,
    ],
    "w3B": [
        1.584,
        1.590,
        1.569,
        1.601,
        1.568,
        1.558,
        1.529,
        1.578,
        1.552,
        1.569,
        1.594,
    ],
    "wHR": [
        2.035,
        2.050,
        2.004,
        2.072,
        2.007,
        1.979,
        1.940,
        2.031,
        1.980,
        2.015,
        2.065,
    ],
    "runSB": [0.200] * 11,
    "runCS": [
        -0.411,
        -0.405,
        -0.422,
        -0.397,
        -0.419,
        -0.435,
        -0.435,
        -0.407,
        -0.423,
        -0.410,
        -0.392,
    ],
    "R/PA": [
        0.119,
        0.117,
        0.122,
        0.114,
        0.121,
        0.125,
        0.126,
        0.117,
        0.122,
        0.118,
        0.112,
    ],
    "R/W": [
        9.808,
        9.683,
        10.028,
        9.524,
        9.973,
        10.282,
        10.296,
        9.714,
        10.048,
        9.778,
        9.421,
    ],
    "cFIP": [
        3.146,
        3.166,
        3.255,
        3.112,
        3.170,
        3.191,
        3.214,
        3.160,
        3.158,
        3.147,
        3.134,
    ],
}

woba_values = pd.DataFrame(woba_values)
woba_values

,Season,wOBA,wOBAScale,wBB,wHBP,w1B,w2B,w3B,wHR,runSB,runCS,R/PA,R/W,cFIP
0,2025,0.314,1.230,0.693,0.724,0.883,1.252,1.584,2.035,0.2,-0.411,0.119,9.808,3.146
1,2024,0.310,1.242,0.689,0.720,0.882,1.254,1.590,2.050,0.2,-0.405,0.117,9.683,3.166
2,2023,0.318,1.204,0.696,0.726,0.883,1.244,1.569,2.004,0.2,-0.422,0.122,10.028,3.255
3,2022,0.310,1.259,0.689,0.720,0.884,1.261,1.601,2.072,0.2,-0.397,0.114,9.524,3.112
4,2021,0.314,1.209,0.692,0.722,0.879,1.242,1.568,2.007,0.2,-0.419,0.121,9.973,3.170
5,2020,0.320,1.185,0.699,0.728,0.883,1.238,1.558,1.979,0.2,-0.435,0.125,10.282,3.191
6,2019,0.320,1.157,0.690,0.719,0.870,1.217,1.529,1.940,0.2,-0.435,0.126,10.296,3.214
7,2018,0.315,1.226,0.690,0.720,0.880,1.247,1.578,2.031,0.2,-0.407,0.117,9.714,3.160
8,2017,0.321,1.185,0.693,0.723,0.877,1.232,1.552,1.980,0.2,-0.423,0.122,10.048,3.158
9,2016,0.318,1.212,0.691,0.721,0.878,1.242,1.569,2.015,0.2,-0.410,0.118,9.778,3.147


## Manually calculating wOBA for more precision
The current wOBA values from StatCast is limited to only 3 decimal points so they're not as precised as they could be. To fix this, we can manually calculate the wOBA values and get more precised values for each player.

Formula: $\frac{(wBB*uBB)+(wHBP*HBP)+(w1B*1B)+(w2B*2B)+(w3B*3B)+(wHR*HR)}{AB+BB-IBB+SF+HBP}$

In [ ]:
def calculate_woba(
    bb: int,
    hbp: int,
    single: int,
    double: int,
    triple: int,
    homerun: int,
    ab: int,
    ibb: int,
    sf: int,
    year: int,
) -> float:

    # Get weights
    wBB = woba_values.loc[woba_values["Season"] == year, "wBB"].iloc[0]
    wHBP = woba_values.loc[woba_values["Season"] == year, "wHBP"].iloc[0]
    w1B = woba_values.loc[woba_values["Season"] == year, "w1B"].iloc[0]
    w2B = woba_values.loc[woba_values["Season"] == year, "w2B"].iloc[0]
    w3B = woba_values.loc[woba_values["Season"] == year, "w3B"].iloc[0]
    wHR = woba_values.loc[woba_values["Season"] == year, "wHR"].iloc[0]

    # print(f"{year}: {wBB}")

    # Numerator
    num = (
        (wBB * (bb - ibb))
        + (wHBP * hbp)
        + (w1B * single)
        + (w2B * double)
        + (w3B * triple)
        + (wHR * homerun)
    )

    # Denominator
    denom = ab + bb - ibb + sf + hbp

    # Final value
    woba = num / denom

    return woba


# Calculate wOBA
df_mlb_stats["wOBA_calculated"] = df_mlb_stats.apply(
    lambda row: calculate_woba(
        row["BB"],
        row["HBP"],
        row["1B"],
        row["2B"],
        row["3B"],
        row["HR"],
        row["AB"],
        row["IBB"],
        row["SF"],
        row["Year"],
    ),
    axis=1,
)

# Display data
df_mlb_stats

,Name,StatCast_ID,Year,Age,AB,PA,1B,2B,3B,HR,...,ISO,BABIP,CS,r_total_stolen_base,HBP,IBB,SF,wOBA,Speed,wOBA_calculated
0,"Ruiz, Carlos",434563,2015,36,284,320,44,13,1,2,...,0.074,0.242,1,1,4,2,1,0.259,25.6,0.258889
1,"García, Avisaíl",541645,2015,24,553,601,110,17,2,13,...,0.108,0.320,7,7,8,3,4,0.295,29.0,0.295502
2,"Dykstra, Allan",488852,2015,28,31,38,3,0,0,1,...,0.097,0.167,0,0,1,0,0,0.251,23.4,0.251263
3,"Nelson, Jimmy",519076,2015,26,55,59,5,1,0,0,...,0.018,0.286,0,0,0,0,0,0.113,23.5,0.113357
4,"Garza, Matt",490063,2015,31,39,47,3,0,0,0,...,0.000,0.167,1,0,0,0,0,0.083,23.6,0.083250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
619,"Steer, Spencer",668715,2025,27,472,524,73,20,2,17,...,0.159,0.279,1,7,4,0,2,0.309,28.0,0.308495
620,"Díaz, Yandy",650490,2025,33,557,618,112,28,1,24,...,0.183,0.315,1,3,6,3,5,0.360,26.1,0.359823
621,"Herrera, Jose",645444,2025,28,166,204,23,6,0,2,...,0.072,0.242,0,0,1,0,3,0.252,24.8,0.251575
622,"Raley, Luke",670042,2025,30,170,203,23,7,0,4,...,0.112,0.275,1,2,11,0,1,0.288,28.7,0.288255


## Calculate wRAA

Formula: $(\frac{wOBA - lgwOBA}{wOBAScale})* PA$


In [ ]:
def calculate_raa(woba: float, pa: float, year: int) -> float:
    league_woba = woba_values.loc[woba_values["Season"] == year, "wOBA"].iloc[0]
    woba_scale = woba_values.loc[woba_values["Season"] == year, "wOBAScale"].iloc[0]
    return ((woba - league_woba) / woba_scale) * pa


# Calculate wRAA
df_mlb_stats["wRAA"] = df_mlb_stats.apply(
    lambda row: calculate_raa(row["wOBA_calculated"], row["PA"], row["Year"]), axis=1
)

# Display data
df_mlb_stats

,Name,StatCast_ID,Year,Age,AB,PA,1B,2B,3B,HR,...,BABIP,CS,r_total_stolen_base,HBP,IBB,SF,wOBA,Speed,wOBA_calculated,wRAA
0,"Ruiz, Carlos",434563,2015,36,284,320,44,13,1,2,...,0.242,1,1,4,2,1,0.259,25.6,0.258889,-13.841371
1,"García, Avisaíl",541645,2015,24,553,601,110,17,2,13,...,0.320,7,7,8,3,4,0.295,29.0,0.295502,-8.406471
2,"Dykstra, Allan",488852,2015,28,31,38,3,0,0,1,...,0.167,0,0,1,0,0,0.251,23.4,0.251263,-1.875300
3,"Nelson, Jimmy",519076,2015,26,55,59,5,1,0,0,...,0.286,0,0,0,0,0,0.113,23.5,0.113357,-9.415610
4,"Garza, Matt",490063,2015,31,39,47,3,0,0,0,...,0.167,1,0,0,0,0,0.083,23.6,0.083250,-8.631695
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
619,"Steer, Spencer",668715,2025,27,472,524,73,20,2,17,...,0.279,1,7,4,0,2,0.309,28.0,0.308495,-2.345126
620,"Díaz, Yandy",650490,2025,33,557,618,112,28,1,24,...,0.315,1,3,6,3,5,0.360,26.1,0.359823,23.023145
621,"Herrera, Jose",645444,2025,28,166,204,23,6,0,2,...,0.242,0,0,1,0,3,0.252,24.8,0.251575,-10.353393
622,"Raley, Luke",670042,2025,30,170,203,23,7,0,4,...,0.275,1,2,11,0,1,0.288,28.7,0.288255,-4.248972


## Calculate wRC

Formula: $(\frac{wOBA - lgwOBA}{wOBAScale} + \frac{lgRun}{PA}) * PA$

In [18]:
def calculate_wrc(woba: float, pa: int, year: int) -> float:
    league_woba = woba_values.loc[woba_values["Season"] == year, "wOBA"].iloc[0]
    woba_scale = woba_values.loc[woba_values["Season"] == year, "wOBAScale"].iloc[0]
    league_runs_per_pa = woba_values.loc[woba_values["Season"] == year, "R/PA"].iloc[0]
    return (((woba - league_woba) / woba_scale) + league_runs_per_pa) * pa


# Calculate wRC
df_mlb_stats["wRC"] = df_mlb_stats.apply(
    lambda row: calculate_wrc(row["wOBA_calculated"], row["PA"], row["Year"]), axis=1
)

# Display data
df_mlb_stats

,Name,StatCast_ID,Year,Age,AB,PA,1B,2B,3B,HR,...,CS,r_total_stolen_base,HBP,IBB,SF,wOBA,Speed,wOBA_calculated,wRAA,wRC
0,"Ruiz, Carlos",434563,2015,36,284,320,44,13,1,2,...,1,1,4,2,1,0.259,25.6,0.258889,-13.841371,21.998629
1,"García, Avisaíl",541645,2015,24,553,601,110,17,2,13,...,7,7,8,3,4,0.295,29.0,0.295502,-8.406471,58.905529
2,"Dykstra, Allan",488852,2015,28,31,38,3,0,0,1,...,0,0,1,0,0,0.251,23.4,0.251263,-1.875300,2.380700
3,"Nelson, Jimmy",519076,2015,26,55,59,5,1,0,0,...,0,0,0,0,0,0.113,23.5,0.113357,-9.415610,-2.807610
4,"Garza, Matt",490063,2015,31,39,47,3,0,0,0,...,1,0,0,0,0,0.083,23.6,0.083250,-8.631695,-3.367695
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
619,"Steer, Spencer",668715,2025,27,472,524,73,20,2,17,...,1,7,4,0,2,0.309,28.0,0.308495,-2.345126,60.010874
620,"Díaz, Yandy",650490,2025,33,557,618,112,28,1,24,...,1,3,6,3,5,0.360,26.1,0.359823,23.023145,96.565145
621,"Herrera, Jose",645444,2025,28,166,204,23,6,0,2,...,0,0,1,0,3,0.252,24.8,0.251575,-10.353393,13.922607
622,"Raley, Luke",670042,2025,30,170,203,23,7,0,4,...,1,2,11,0,1,0.288,28.7,0.288255,-4.248972,19.908028


## Get League Stats and Park Factors

### NL Values

In [20]:
# NL Batting Data
nl_data = [
    {
        "Season": 2025,
        "PA": 86294,
        "BB%": 8.6,
        "K%": 21.8,
        "BB/K": 0.39,
        "AVG": 0.247,
        "OBP": 0.318,
        "SLG": 0.402,
        "OPS": 0.720,
        "ISO": 0.155,
        "Spd": 4.9,
        "BABIP": 0.293,
        "UBR": None,
        "wGDP": None,
        "XBR": 13.6,
        "wSB": -3.2,
        "wRC": 10245,
        "wRAA": 0.7,
        "wOBA": 0.314,
        "wRC+": 100,
    },
    {
        "Season": 2024,
        "PA": 91611,
        "BB%": 8.2,
        "K%": 22.6,
        "BB/K": 0.36,
        "AVG": 0.247,
        "OBP": 0.315,
        "SLG": 0.404,
        "OPS": 0.719,
        "ISO": 0.157,
        "Spd": 5.1,
        "BABIP": 0.295,
        "UBR": 30.2,
        "wGDP": 4.2,
        "XBR": 37.3,
        "wSB": 25.8,
        "wRC": 10955,
        "wRAA": 238.4,
        "wOBA": 0.313,
        "wRC+": 100,
    },
    {
        "Season": 2023,
        "PA": 92210,
        "BB%": 8.8,
        "K%": 22.3,
        "BB/K": 0.39,
        "AVG": 0.250,
        "OBP": 0.323,
        "SLG": 0.417,
        "OPS": 0.740,
        "ISO": 0.166,
        "Spd": 5.0,
        "BABIP": 0.297,
        "UBR": 21.3,
        "wGDP": -6.3,
        "XBR": 18.5,
        "wSB": 16.2,
        "wRC": 11424,
        "wRAA": 188.8,
        "wOBA": 0.321,
        "wRC+": 100,
    },
    {
        "Season": 2022,
        "PA": 91188,
        "BB%": 8.4,
        "K%": 22.6,
        "BB/K": 0.37,
        "AVG": 0.243,
        "OBP": 0.314,
        "SLG": 0.398,
        "OPS": 0.712,
        "ISO": 0.155,
        "Spd": 4.5,
        "BABIP": 0.291,
        "UBR": 24.3,
        "wGDP": 2.5,
        "XBR": -4.2,
        "wSB": 3.7,
        "wRC": 10596,
        "wRAA": 169.0,
        "wOBA": 0.312,
        "wRC+": 100,
    },
    {
        "Season": 2021,
        "PA": 86485,
        "BB%": 9.2,
        "K%": 22.3,
        "BB/K": 0.41,
        "AVG": 0.249,
        "OBP": 0.326,
        "SLG": 0.420,
        "OPS": 0.747,
        "ISO": 0.171,
        "Spd": 4.4,
        "BABIP": 0.295,
        "UBR": -4.6,
        "wGDP": 5.7,
        "XBR": 4.0,
        "wSB": -9.8,
        "wRC": 11010,
        "wRAA": 540.5,
        "wOBA": 0.322,
        "wRC+": 100,
    },
    {
        "Season": 2020,
        "PA": 33198,
        "BB%": 9.3,
        "K%": 23.1,
        "BB/K": 0.40,
        "AVG": 0.246,
        "OBP": 0.325,
        "SLG": 0.421,
        "OPS": 0.746,
        "ISO": 0.175,
        "Spd": 4.6,
        "BABIP": 0.293,
        "UBR": -9.3,
        "wGDP": -4.8,
        "XBR": 2.0,
        "wSB": -1.4,
        "wRC": 4242,
        "wRAA": 76.6,
        "wOBA": 0.322,
        "wRC+": 100,
    },
    {
        "Season": 2019,
        "PA": 88387,
        "BB%": 8.9,
        "K%": 21.9,
        "BB/K": 0.41,
        "AVG": 0.258,
        "OBP": 0.331,
        "SLG": 0.445,
        "OPS": 0.776,
        "ISO": 0.187,
        "Spd": 4.5,
        "BABIP": 0.300,
        "UBR": 3.1,
        "wGDP": -8.3,
        "XBR": 9.8,
        "wSB": 8.2,
        "wRC": 11661,
        "wRAA": 540.8,
        "wOBA": 0.327,
        "wRC+": 100,
    },
    {
        "Season": 2018,
        "PA": 88080,
        "BB%": 9.0,
        "K%": 21.6,
        "BB/K": 0.42,
        "AVG": 0.254,
        "OBP": 0.327,
        "SLG": 0.417,
        "OPS": 0.744,
        "ISO": 0.163,
        "Spd": 4.4,
        "BABIP": 0.301,
        "UBR": 33.0,
        "wGDP": 8.3,
        "XBR": 31.9,
        "wSB": -9.3,
        "wRC": 10752,
        "wRAA": 461.6,
        "wOBA": 0.321,
        "wRC+": 100,
    },
    {
        "Season": 2017,
        "PA": 87753,
        "BB%": 9.0,
        "K%": 21.0,
        "BB/K": 0.43,
        "AVG": 0.261,
        "OBP": 0.334,
        "SLG": 0.437,
        "OPS": 0.771,
        "ISO": 0.176,
        "Spd": 4.4,
        "BABIP": 0.305,
        "UBR": 5.4,
        "wGDP": 7.7,
        "XBR": 17.4,
        "wSB": -4.2,
        "wRC": 11280,
        "wRAA": 585.4,
        "wOBA": 0.329,
        "wRC+": 100,
    },
    {
        "Season": 2016,
        "PA": 87545,
        "BB%": 8.6,
        "K%": 20.5,
        "BB/K": 0.42,
        "AVG": 0.260,
        "OBP": 0.330,
        "SLG": 0.425,
        "OPS": 0.756,
        "ISO": 0.165,
        "Spd": 4.7,
        "BABIP": 0.305,
        "UBR": 20.2,
        "wGDP": 19.3,
        "XBR": 30.8,
        "wSB": 3.5,
        "wRC": 10782,
        "wRAA": 468.8,
        "wOBA": 0.325,
        "wRC+": 100,
    },
    {
        "Season": 2015,
        "PA": 86736,
        "BB%": 7.9,
        "K%": 19.9,
        "BB/K": 0.40,
        "AVG": 0.260,
        "OBP": 0.324,
        "SLG": 0.410,
        "OPS": 0.734,
        "ISO": 0.150,
        "Spd": 4.6,
        "BABIP": 0.306,
        "UBR": 41.4,
        "wGDP": -18.7,
        "XBR": None,
        "wSB": 15.5,
        "wRC": 10107,
        "wRAA": 354.0,
        "wOBA": 0.318,
        "wRC+": 100,
    },
]

# Create the pandas DataFrame from the list of dictionaries
nl_data_df = pd.DataFrame(nl_data)

# Display
nl_data_df

,Season,PA,BB%,K%,BB/K,AVG,OBP,SLG,OPS,ISO,Spd,BABIP,UBR,wGDP,XBR,wSB,wRC,wRAA,wOBA,wRC+
0,2025,86294,8.6,21.8,0.39,0.247,0.318,0.402,0.720,0.155,4.9,0.293,NaN,NaN,13.6,-3.2,10245,0.7,0.314,100
1,2024,91611,8.2,22.6,0.36,0.247,0.315,0.404,0.719,0.157,5.1,0.295,30.2,4.2,37.3,25.8,10955,238.4,0.313,100
2,2023,92210,8.8,22.3,0.39,0.250,0.323,0.417,0.740,0.166,5.0,0.297,21.3,-6.3,18.5,16.2,11424,188.8,0.321,100
3,2022,91188,8.4,22.6,0.37,0.243,0.314,0.398,0.712,0.155,4.5,0.291,24.3,2.5,-4.2,3.7,10596,169.0,0.312,100
4,2021,86485,9.2,22.3,0.41,0.249,0.326,0.420,0.747,0.171,4.4,0.295,-4.6,5.7,4.0,-9.8,11010,540.5,0.322,100
5,2020,33198,9.3,23.1,0.40,0.246,0.325,0.421,0.746,0.175,4.6,0.293,-9.3,-4.8,2.0,-1.4,4242,76.6,0.322,100
6,2019,88387,8.9,21.9,0.41,0.258,0.331,0.445,0.776,0.187,4.5,0.300,3.1,-8.3,9.8,8.2,11661,540.8,0.327,100
7,2018,88080,9.0,21.6,0.42,0.254,0.327,0.417,0.744,0.163,4.4,0.301,33.0,8.3,31.9,-9.3,10752,461.6,0.321,100
8,2017,87753,9.0,21.0,0.43,0.261,0.334,0.437,0.771,0.176,4.4,0.305,5.4,7.7,17.4,-4.2,11280,585.4,0.329,100
9,2016,87545,8.6,20.5,0.42,0.260,0.330,0.425,0.756,0.165,4.7,0.305,20.2,19.3,30.8,3.5,10782,468.8,0.325,100


### AL Values

In [22]:
import pandas as pd

al_data = {
    "Season": [2025, 2024, 2023, 2022, 2021, 2020, 2019, 2018, 2017, 2016, 2015],
    "PA": [85837, 90836, 91884, 90850, 90544, 33304, 92956, 91924, 92265, 91672, 91485],
    "BB%": [
        8.3,
        8.1,
        8.4,
        8.0,
        8.5,
        9.0,
        8.5,
        8.3,
        8.4,
        8.0,
        7.7,
    ],
    "K%": [
        22.3,
        22.6,
        23.1,
        22.2,
        22.9,
        23.8,
        22.9,
        21.8,
        21.3,
        20.7,
        19.8,
    ],
    "BB/K": [0.37, 0.36, 0.36, 0.36, 0.37, 0.38, 0.37, 0.38, 0.40, 0.39, 0.39],
    "AVG": [
        0.245,
        0.240,
        0.247,
        0.242,
        0.246,
        0.243,
        0.254,
        0.250,
        0.256,
        0.257,
        0.256,
    ],
    "OBP": [
        0.314,
        0.309,
        0.317,
        0.309,
        0.317,
        0.319,
        0.323,
        0.319,
        0.325,
        0.322,
        0.318,
    ],
    "SLG": [
        0.407,
        0.394,
        0.412,
        0.392,
        0.416,
        0.414,
        0.440,
        0.417,
        0.430,
        0.424,
        0.413,
    ],
    "OPS": [
        0.722,
        0.703,
        0.729,
        0.701,
        0.732,
        0.733,
        0.763,
        0.735,
        0.754,
        0.746,
        0.732,
    ],
    "ISO": [
        0.163,
        0.155,
        0.165,
        0.149,
        0.170,
        0.171,
        0.186,
        0.167,
        0.174,
        0.166,
        0.157,
    ],
    "Spd": [4.6, 4.6, 4.9, 4.2, 4.3, 4.3, 4.4, 4.5, 4.2, 4.3, 4.3],
    "BABIP": [
        0.289,
        0.286,
        0.296,
        0.290,
        0.292,
        0.291,
        0.299,
        0.294,
        0.298,
        0.299,
        0.296,
    ],
    "UBR": [None, -29.0, -21.4, -22.2, 4.6, 9.7, -3.6, -32.0, -4.8, -19.9, -41.5],
    "wGDP": [None, -5.6, 4.9, -2.4, -6.2, 4.7, 6.8, -8.3, -8.8, -19.7, 18.1],
    "XBR": [0.8, -38.9, -11.7, 4.2, 6.3, -9.0, -1.8, -17.1, -11.4, -14.3, None],
    "wSB": [3.2, -25.8, -16.1, -3.7, 11.3, 1.4, -6.4, 9.3, 5.4, -2.2, 13.8],
    "wRC": [10195, 10394, 11010, 10223, 11086, 4103, 11881, 10989, 11384, 11007, 10587],
    "wRAA": [
        4.2,
        -232.0,
        -185.9,
        -165.0,
        124.8,
        -75.6,
        185.2,
        249.5,
        139.3,
        207.2,
        300.0,
    ],
    "wOBA": [
        0.314,
        0.307,
        0.316,
        0.307,
        0.316,
        0.317,
        0.322,
        0.318,
        0.323,
        0.321,
        0.317,
    ],
    "wRC+": [100, 101, 101, 100, 100, 100, 100, 100, 100, 100, 100],
}

al_data_df = pd.DataFrame(al_data)
al_data_df

,Season,PA,BB%,K%,BB/K,AVG,OBP,SLG,OPS,ISO,Spd,BABIP,UBR,wGDP,XBR,wSB,wRC,wRAA,wOBA,wRC+
0,2025,85837,8.3,22.3,0.37,0.245,0.314,0.407,0.722,0.163,4.6,0.289,NaN,NaN,0.8,3.2,10195,4.2,0.314,100
1,2024,90836,8.1,22.6,0.36,0.240,0.309,0.394,0.703,0.155,4.6,0.286,-29.0,-5.6,-38.9,-25.8,10394,-232.0,0.307,101
2,2023,91884,8.4,23.1,0.36,0.247,0.317,0.412,0.729,0.165,4.9,0.296,-21.4,4.9,-11.7,-16.1,11010,-185.9,0.316,101
3,2022,90850,8.0,22.2,0.36,0.242,0.309,0.392,0.701,0.149,4.2,0.290,-22.2,-2.4,4.2,-3.7,10223,-165.0,0.307,100
4,2021,90544,8.5,22.9,0.37,0.246,0.317,0.416,0.732,0.170,4.3,0.292,4.6,-6.2,6.3,11.3,11086,124.8,0.316,100
5,2020,33304,9.0,23.8,0.38,0.243,0.319,0.414,0.733,0.171,4.3,0.291,9.7,4.7,-9.0,1.4,4103,-75.6,0.317,100
6,2019,92956,8.5,22.9,0.37,0.254,0.323,0.440,0.763,0.186,4.4,0.299,-3.6,6.8,-1.8,-6.4,11881,185.2,0.322,100
7,2018,91924,8.3,21.8,0.38,0.250,0.319,0.417,0.735,0.167,4.5,0.294,-32.0,-8.3,-17.1,9.3,10989,249.5,0.318,100
8,2017,92265,8.4,21.3,0.40,0.256,0.325,0.430,0.754,0.174,4.2,0.298,-4.8,-8.8,-11.4,5.4,11384,139.3,0.323,100
9,2016,91672,8.0,20.7,0.39,0.257,0.322,0.424,0.746,0.166,4.3,0.299,-19.9,-19.7,-14.3,-2.2,11007,207.2,0.321,100


Note that the FanGraph Park Factors are "halved" because they only factor in home games (and players half of their games at home). FanGraph assumes "the rest [of the away games] are played in a pretty average setting."

## Calculate wRC+

Formula: $\frac{(wRAA/PA + lgRun/PA) + (lgRun/PA - (ParkFactor * lgRun/PA))}{wRC^*/PA} * PA$

where $\frac{wRC^*}{PA}$ = AL or NL $\frac{wRC}{PA}$ excluding pitchers

In [ ]:
def calculate_wrc_plus(wraa: float, pa: int, year: int) -> float:
    return 0